In [ ]:

import pandas as pd
import numpy as np
from datetime import datetime, date
from io import BytesIO
from ipywidgets import (
    FileUpload, Button, VBox, Output, Label, SelectMultiple, HBox,
    FloatSlider, Dropdown, Text, Checkbox
)
from IPython.display import display, clear_output
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import objective_functions
import copy

output_all = Output()
df_global = None
selected_assets = []
sector_mapper = {}
sector_constraint_widgets = {}
expected_return_inputs = {}
constraint_rows = []
constraint_inputs = []


In [ ]:

def log_returns(df):
    return np.log(df / df.shift(1)).dropna()

def ewma_covariance_matrix(X, alpha):
    T, D = X.shape
    cov = np.zeros((D, D))
    for tt in range(T):
        x_t = X[tt, :]
        if tt == 0:
            cov = x_t[:, np.newaxis] @ x_t[np.newaxis, :]
        else:
            cov = alpha * cov + (1 - alpha) * x_t[:, np.newaxis] @ x_t[np.newaxis, :]
    return cov

def make_equal_constraint(i, v): return lambda x: x[i] == v
def make_le_constraint(i, v): return lambda x: x[i] <= v
def make_ge_constraint(i, v): return lambda x: x[i] >= v
def make_equal_pair_constraint(i, j): return lambda x: x[i] == x[j]
def make_le_pair_constraint(i, j): return lambda x: x[i] <= x[j]
def make_ge_pair_constraint(i, j): return lambda x: x[i] >= x[j]


In [ ]:

def show_asset_selector():
    with output_all:
        clear_output()
        asset_select = SelectMultiple(description='Assets:', options=df_global.columns.tolist(), rows=10)
        continue_btn = Button(description='➡ Define Sectors', button_style='primary')

        def proceed(change):
            global selected_assets
            selected_assets = list(asset_select.value)
            if not selected_assets:
                print("⚠️ Please select at least one asset.")
                return
            show_expected_return_inputs()

        continue_btn.on_click(proceed)
        display(VBox([
            Label("📊 Step 2: Select assets:"),
            asset_select,
            continue_btn
        ]))


In [ ]:

def show_expected_return_inputs():
    with output_all:
        clear_output()
        global expected_return_inputs
        expected_return_inputs = {}
        rows = []

        for name in selected_assets:
            label = Label(value=name, layout={'width': '200px'})
            er_input = Text(placeholder='Expected Return (e.g. 0.05)', layout={'width': '100px'})
            expected_return_inputs[name] = er_input
            rows.append(HBox([label, er_input]))

        next_btn = Button(description="➡ Define Sectors", button_style="primary")

        def proceed(change):
            for asset, widget in expected_return_inputs.items():
                try:
                    float(widget.value)
                except ValueError:
                    print(f"❌ Invalid input for {asset}. Please enter a numeric value.")
                    return
            show_expected_return_summary()

        next_btn.on_click(proceed)

        display(VBox([
            Label("📈 Step 3: Enter expected returns:"),
            VBox(rows),
            next_btn
        ]))

def show_expected_return_summary():
    with output_all:
        clear_output()
        summary = pd.DataFrame.from_dict(
            {k: [float(v.value)] for k, v in expected_return_inputs.items()},
            orient='index',
            columns=['Expected Return']
        )
        print("✅ Expected Returns Summary:")
        display(summary)

        next_btn = Button(description="➡ Define Sectors", button_style="primary")
        next_btn.on_click(lambda change: show_sector_mapping())
        display(next_btn)


In [ ]:

def show_sector_mapping():
    with output_all:
        clear_output()
        mapping_widgets = {}
        rows = []

        for asset in selected_assets:
            label = Label(value=asset, layout={"width": "200px"})
            dropdown = Dropdown(
                options=["Equities", "Corporate Bonds", "High Yield", "Euro Government Core", "Cash"],
                layout={"width": "200px"}
            )
            mapping_widgets[asset] = dropdown
            rows.append(HBox([label, dropdown]))

        next_btn = Button(description="➡ Set Sector Constraints", button_style="primary")

        def proceed_mapping(change):
            global sector_mapper
            sector_mapper = {a: w.value for a, w in mapping_widgets.items()}
            show_sector_mapping_summary()

        next_btn.on_click(proceed_mapping)
        display(VBox([
            Label("🏷 Step 4: Assign sectors to assets:"),
            VBox(rows),
            next_btn
        ]))

def show_sector_mapping_summary():
    with output_all:
        clear_output()
        summary = pd.DataFrame.from_dict(sector_mapper, orient='index', columns=['Sector'])
        print("✅ Sector Mapping Summary:")
        display(summary)

        next_btn = Button(description="➡ Set Sector Constraints", button_style="primary")
        next_btn.on_click(lambda change: show_additional_constraints())
        display(next_btn)


In [ ]:

def show_additional_constraints():
    with output_all:
        clear_output()
        constraint_rows.clear()
        constraint_inputs.clear()

        for _ in range(3):  # up to 3 constraints
            asset1_dd = Dropdown(options=selected_assets, description="Asset A")
            relation_dd = Dropdown(options=["==", "<=", ">="], description="Relation")
            asset2_dd = Dropdown(options=selected_assets + ["Value"], description="Asset B/Value")
            value_input = Text(placeholder="0.1")

            constraint_inputs.append((asset1_dd, relation_dd, asset2_dd, value_input))
            row = HBox([asset1_dd, relation_dd, asset2_dd, value_input])
            constraint_rows.append(row)

        next_btn = Button(description="📋 Review and Confirm", button_style="primary")
        next_btn.on_click(lambda change: show_review_and_confirm())

        display(VBox([
            Label("🧩 Additional Constraints (A <= B or A == 0.1):"),
            VBox(constraint_rows),
            next_btn
        ]))

def apply_additional_constraints(ef, var_names):
    for asset1_dd, relation_dd, asset2_dd, value_input in constraint_inputs:
        a1 = asset1_dd.value
        rel = relation_dd.value
        a2 = asset2_dd.value
        if not a1 or not a2:
            continue

        idx1 = var_names.index(a1)

        if a2 == "Value":
            try:
                val = float(value_input.value)
            except:
                continue

            if rel == "==":
                ef.add_constraint(make_equal_constraint(idx1, val))
            elif rel == "<=":
                ef.add_constraint(make_le_constraint(idx1, val))
            elif rel == ">=":
                ef.add_constraint(make_ge_constraint(idx1, val))
        else:
            try:
                idx2 = var_names.index(a2)
                if rel == "==":
                    ef.add_constraint(make_equal_pair_constraint(idx1, idx2))
                elif rel == "<=":
                    ef.add_constraint(make_le_pair_constraint(idx1, idx2))
                elif rel == ">=":
                    ef.add_constraint(make_ge_pair_constraint(idx1, idx2))
            except:
                continue

def show_review_and_confirm():
    with output_all:
        clear_output()
        print("🔎 Final Review and Confirm")

        print("✅ Assets Selected:", selected_assets)
        print("\n✅ Expected Returns:")
        display(pd.DataFrame.from_dict({k: [float(v.value)] for k, v in expected_return_inputs.items()},
                                       orient='index', columns=['Expected Return']))

        print("\n✅ Sector Mapping:")
        display(pd.DataFrame.from_dict(sector_mapper, orient='index', columns=['Sector']))

        print("\n✅ Constraints:")
        for asset1, relation, asset2, value in constraint_inputs:
            print(f"{asset1.value} {relation.value} {asset2.value if asset2.value != 'Value' else value.value}")

        run_btn = Button(description="🚀 Run Optimization", button_style="success")
        run_btn.on_click(run_optimizer)
        display(run_btn)

def run_optimizer(change):
    with output_all:
        clear_output()
        try:
            df = df_global[selected_assets]
            logret = log_returns(df).astype(np.float64)
            cov = ewma_covariance_matrix(logret.values, alpha=0.99)
            cov = pd.DataFrame(cov * 50, index=df.columns, columns=df.columns)

            mu = {asset: float(expected_return_inputs[asset].value.strip() or 0) for asset in selected_assets}
            mu = pd.Series(mu)

            ef = EfficientFrontier(mu, cov, weight_bounds=(0, 1))
            ef.add_objective(objective_functions.L2_reg, gamma=0.01)

            sector_lower = {s: 0.0 for s in set(sector_mapper.values())}
            sector_upper = {s: 1.0 for s in set(sector_mapper.values())}
            ef.add_sector_constraints(sector_mapper, sector_lower, sector_upper)

            apply_additional_constraints(ef, selected_assets)

            return_range = np.linspace(ef.expected_returns.min(), ef.expected_returns.max() - 0.001, 25)
            weights = []
            metrics = []

            for r in return_range:
                ef_copy = copy.deepcopy(ef)
                ef_copy.efficient_return(r)
                weights.append(ef_copy.weights)
                metrics.append(ef_copy.portfolio_performance())

            weights_df = pd.DataFrame(weights, columns=mu.index)
            weights_df["Target Return"] = return_range
            metrics_df = pd.DataFrame(metrics, columns=["Mean", "Volatility", "Sharpe"])

            display(weights_df.head())
            display(metrics_df.head())

            today = date.today().strftime("%d%m%y")
            weights_df.to_excel(f"efficient_frontier_weights_{today}.xlsx", index=False)
            metrics_df.to_excel(f"efficient_frontier_metrics_{today}.xlsx", index=False)
            print(f"✅ Files saved: efficient_frontier_weights_{today}.xlsx, efficient_frontier_metrics_{today}.xlsx")

        except Exception as e:
            print("❌ Error:", e)


In [ ]:

def handle_upload(change):
    global df_global
    with output_all:
        clear_output()
        try:
            file_info = upload.value[0]
            content = BytesIO(file_info["content"])
            df = pd.read_excel(content, sheet_name="PyData", engine="openpyxl")
            df = df.rename(columns=df.iloc[0]).drop(0)
            df["Dates"] = pd.to_datetime(df["Dates"])
            df = df.set_index("Dates")
            df = df.astype(float)
            df = df[df.index > datetime(2017, 1, 1)]
            df_global = df
            print("✅ File loaded successfully.")
            show_asset_selector()
        except Exception as e:
            print("❌ Error loading file:", e)

upload = FileUpload(accept='.xlsx', multiple=False)
upload.observe(handle_upload, names='value')
display(VBox([
    Label('📥 Step 1: Upload Excel file (sheet: PyData):'),
    upload,
    output_all
]))
